In [ ]:
!pip install boto3

In [2]:
import utils as u
import os

In [3]:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

In [4]:
aws_separator = "/"

In [15]:
# Import S3 connectors librairies
#from awsPy.aws_authorization import aws_connector
import importlib.util
spec = importlib.util.spec_from_file_location("aws_connector.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_authorization/aws_connector.py")
aws_connector = importlib.util.module_from_spec(spec)
spec.loader.exec_module(aws_connector)

#from awsPy.aws_s3 import service_s3
import importlib.util
spec = importlib.util.spec_from_file_location("service_s3.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_s3/service_s3.py")
service_s3 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(service_s3)

In [16]:
# Connect to S3
import os
from pathlib import Path

bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

con = aws_connector.aws_instantiate(credential = path_cred,
                                        region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata')

In [ ]:
# List files to send and compute each destination
prep_path='data/prep'

list_files=[]
for root,dirs,files in os.walk(prep_path):
    for file in files:
          if file.endswith(('.csv')):
                print(file)
                list_files.append((file, u.calc_dest(filename=file)))
list_files[:5]

In [ ]:
# Send files to s3
for (file_to_upload,destination_in_s3) in list_files:
    # if file exist in S3?
    full_path = os.path.join(prep_path, file_to_upload)
    s3.upload_file(full_path, destination_in_s3)

In [ ]:
def file_exists_s3(full_path):
    try:
        client["resource"].Object(bucket, 'dootdoot.jpg').load()
    except Exception as e:
        if e.response['Error']['Code'] == "404":
            # The object does not exist.
            return False
        else:
            # Something else has gone wrong.
            raise
    else:
        # The object does exist.
        return True

In [ ]:
# A FAIRE
# if file exist in S3?
(file_to_upload,destination_in_s3)=list_files[0]
full_path="{}/{}".format(destination_in_s3,file_to_upload)
full_path

In [ ]:
# check file existence in S3
file_exists_s3(full_path)

In [ ]:
# Send Raw files to S3

In [5]:
# List files to send and compute each destination
import os
prep_path= 'flux' #os.path.join(workbookDir,'flux')
root_path=aws_separator.join(['INPI', 'TC_1' , '00_RawData', 'public' ,'IMR_Donnees_Saisies' ,'tc'])

In [6]:
# TEST
root_path

'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc'

In [23]:
list_files=[]
for root,dirs,files in os.walk(prep_path):
    for i, file in enumerate(files):
          if file.endswith(('.csv')):
                #print(root)
                
                source_full_path=os.path.join(root,file)
                #print(source_full_path)
                
                dest_path = aws_separator.join([root_path,root.replace("\\","/")])
                #print(dest_path)
                list_files.append((source_full_path,dest_path))
list_files[:5]

[('flux\\2017\\05\\18\\0101\\1\\0101_1_20170512_112544_10_ets_supprime_EVT.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1'),
 ('flux\\2017\\05\\18\\0101\\1\\0101_1_20170512_112544_11_obs.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1'),
 ('flux\\2017\\05\\18\\0101\\1\\0101_1_20170512_112544_12_actes.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1'),
 ('flux\\2017\\05\\18\\0101\\1\\0101_1_20170512_112544_13_comptes_annuels.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1'),
 ('flux\\2017\\05\\18\\0101\\1\\0101_1_20170512_112544_1_PM.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1')]

In [ ]:
#TEST
#Celui qui fonctionne
(full_path, destination_in_s3)=(r'C:/Users/Hp/Documents/GitHub/InseeInpi_matching/Notebooks_matching/Data_preprocessed/programme_matching/flux/2017/05/18/0101/1/0101_1_20170512_112544_10_ets_supprime_EVT.csv',
 r'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1')
s3.upload_file(full_path, destination_in_s3)

In [19]:
#TEST
list_files[:2]

[('flux\\2017\\05\\18\\0101\\1\\0101_1_20170512_112544_10_ets_supprime_EVT.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1'),
 ('flux\\2017\\05\\18\\0101\\1\\0101_1_20170512_112544_11_obs.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1')]

In [24]:
# Send files to s3 (Raw flux/2017)
#list_files[:100]
for (full_path,destination_in_s3) in list_files:
    # if file exist in S3?
    s3.upload_file(full_path, destination_in_s3)